# Merchant Ranking Algorith
The method of ranking each merchant, will take inspiration from the methods proposed in (https://sapinsider.org/leveraging-analytical-method-for-ranking-suppliers/), in which we rank each key attribute out of 5, then sum them together with weightsget a score for each merchant. 

The Key Attributes for now are: 
- Merchant's BNPL Revenue
- Proportion of 'identified' Fraud transactions
- The customer Base 
- Projected Growth

In [1]:
from pyspark.sql import SparkSession, functions as F
# First lets reed the datasets
spark = (
    SparkSession.builder.appName("Data_Explorer")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/10/06 01:20:44 WARN Utils: Your hostname, James-N580VD-DM229T resolves to a loopback address: 127.0.1.1; using 172.23.63.37 instead (on interface eth0)
22/10/06 01:20:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 01:20:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1. Tansaction_Revenue
For this we will take:
- Total_BNPL_Revenue
- Average_BNPL_Revenue

In [20]:
transactions = spark.read.parquet('../data/processed/transactions')
merchants =  spark.read.parquet('../data/processed/merchants/')
full_dataset = spark.read.parquet('../data/curated/full_dataset/')
final_data_collection = merchants.select('merchant_abn')

In [27]:
full_dataset.select(F.max('order_datetime'))

max(order_datetime)
2021-08-27


In [25]:
final_data_collection = final_data_collection.join(full_dataset.groupBy('merchant_abn').agg(F.round(F.sum('BNPL_Revenue'), 2).alias('Total_BNPL_Revenue'), F.round(F.sum('dollar_value'), 2).alias('Total_Dollar_value'), F.count('BNPL_Revenue').alias('Total_Transactions')), on=['merchant_abn'])
# now add last 6 months
full_dataset.where(F.col('order_datetime') > F.lit('2022-03-01')).groupBy('merchant_abn').agg(F.round(F.sum('BNPL_Revenue'), 2).alias('Total_BNPL_Revenue'), F.round(F.sum('dollar_value'), 2).alias('Total_Dollar_value'), F.count('BNPL_Revenue').alias('Total_Transactions'))


merchant_abn,Total_BNPL_Revenue,Total_Dollar_value,Total_Transactions


In [23]:
final_data_collection

merchant_abn,Total_BNPL_Revenue,Total_Dollar_value,Total_Transactions,Total_BNPL_Revenue,Total_Dollar_value,Total_Transactions
